# Import libraries

In [1]:
import pandas as pd
import numpy as np
import xlwings as xw
import matplotlib.pyplot as plt

# Define global variables

In [15]:
xlDataName = '\DissData [02]'
dataFolder = r"C:\Users\rfg\OneDrive\Desktop\Dissertation ES30029\data"
companyNamesRange = "A1:A2009"
cellRng = "A1:BYH288"
xlBook = xw.books(xlDataName[1:])
sheetWhitelist = ['all_companies']
fields = []

# Read data from Excel

In [16]:
# Requires file to be open, but is SO MUCH FASTER
# p = pd.DataFrame(xw.books(xlDataName[1:]).sheets("p").range(cellRng).value)
# pe = pd.DataFrame(xw.books(xlDataName[1:]).sheets("pe").range(cellRng).value)
# fcf = pd.DataFrame(xw.books(xlDataName[1:]).sheets("fcf").range(cellRng).value)
# eps = pd.DataFrame(xw.books(xlDataName[1:]).sheets("eps").range(cellRng).value)
# ptbv = pd.DataFrame(xw.books(xlDataName[1:]).sheets("ptbv").range(cellRng).value)
# dy = pd.DataFrame(xw.books(xlDataName[1:]).sheets("dy").range(cellRng).value)
# mv = pd.DataFrame(xw.books(xlDataName[1:]).sheets("mv").range(cellRng).value)
# gpmarg = pd.DataFrame(xw.books(xlDataName[1:]).sheets("gpmarg").range(cellRng).value)
# roe = pd.DataFrame(xw.books(xlDataName[1:]).sheets("roe").range(cellRng).value)
# roic = pd.DataFrame(xw.books(xlDataName[1:]).sheets("roic").range(cellRng).value)
# opmarg = pd.DataFrame(xw.books(xlDataName[1:]).sheets("opmarg").range(cellRng).value)
# debtpct = pd.DataFrame(xw.books(xlDataName[1:]).sheets("debtpct").range(cellRng).value)

for sheet in xlBook.sheets:
    if sheet.name not in sheetWhitelist:
        exec('{} = pd.DataFrame(xw.books(xlDataName[1:]).sheets("{}").range(cellRng).value)'.format(sheet.name, sheet.name))
        exec('fields.append({})'.format(sheet.name))
        


# Clean up the dataframes to have correct cols and rows
for field in fields:
    companyNames = [str(i) for i in xlBook.sheets("all_companies").range(companyNamesRange).value]
    companyNames.insert(0, field.iloc[0][0])
    field.columns = companyNames
    field.index = field[field.columns[0]]
    del field[field.columns[0]]
    field.drop(field.index[0], inplace=True)
    field.replace("NA", np.nan, inplace=True)
    field = field.apply(pd.to_numeric,errors='coerce')
    


# Data Cleaning

### Remove series with no price data

In [17]:
# There are 16 series of this sort
errorSeries = []
for i in p.columns:
    try:
        m = max(p[i])
    except:
        errorSeries.append(i)

In [18]:
for field in fields:
    for series in errorSeries:
        del field[series]

# Data Calculation

### Returns

In [19]:
r = p/p.shift(1)-1

In [20]:
ret_l3m = (1+r).rolling(window=3).apply(np.prod, raw=True)-1
ret_l6m = (1+r).rolling(window=6).apply(np.prod, raw=True)-1
ret_l9m = (1+r).rolling(window=9).apply(np.prod, raw=True)-1
ret_l12m = (1+r).rolling(window=12).apply(np.prod, raw=True)-1
ret_l18m = (1+r).rolling(window=18).apply(np.prod, raw=True)-1
ret_l24m = (1+r).rolling(window=24).apply(np.prod, raw=True)-1
ret_l36m = (1+r).rolling(window=36).apply(np.prod, raw=True)-1

### Moving averages

In [21]:
p_ma3 = p.rolling(window=3).mean()
p_ma6 = p.rolling(window=6).mean()
p_ma9 = p.rolling(window=9).mean()
p_ma12 = p.rolling(window=12).mean()
p_ma18 = p.rolling(window=18).mean()
p_ma24 = p.rolling(window=24).mean()
p_ma36 = p.rolling(window=36).mean()

### Volatilities

In [ ]:
std_3m = r.rolling(window=3).std() * (12 ** 0.5)
std_6m = r.rolling(window=6).std() * (12 ** 0.5)
std_9m = r.rolling(window=9).std() * (12 ** 0.5)
std_12m = r.rolling(window=12).std() * (12 ** 0.5)
std_18m = r.rolling(window=18).std() * (12 ** 0.5)
std_24m = r.rolling(window=24).std() * (12 ** 0.5)
std_36m = r.rolling(window=36).std() * (12 ** 0.5)

# Export Data

### Numpy file

In [ ]:
# Also save index and columns so that you can load up data with these
np.save(file=dataFolder+r'\npData.npy', arr=p)
np.save(file=dataFolder+'\index.npy', arr=p.index)
np.save(file=dataFolder+'\columns.npy', arr=p.columns)

In [ ]:
np.stack(fields, axis=2).shape